# Models: Biased Generalized Matrix Factorization (Biased-GMF)

In [1]:
%load_ext autoreload
%autoreload 2
BASE_PATH='../..'

In [2]:
import sys
sys.path.append(f'{BASE_PATH}/lib')

import numpy as np
import pandas as pd
from bunch import Bunch

import torch
from torch.utils.data import DataLoader, Subset
from torch.optim import SparseAdam, Adam
from sklearn.metrics import roc_auc_score

import pytorch_common.util as pu
from pytorch_common.modules.fn import Fn
from pytorch_common.callbacks import EarlyStop, \
                                     ReduceLROnPlateau, \
                                     Validation, \
                                     SaveBestModel
from pytorch_common.callbacks.output import Logger, \
                                            MetricsPlotter

from pytorch_common.util import set_device_name, \
                                get_device, \
                                LoggerBuilder

import model as ml
import data.dataset as ds

import metric as mt
import metric.discretizer as dr

import data.plot as pl
import data as dt

import logging
import random

from torchviz import make_dot

/home/adrian/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/adrian/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")


## Setup

In [3]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [4]:
pu.set_device_name('gpu')

In [5]:
pu.get_device()

device(type='cuda', index=0)

In [6]:
cpu = torch.device("cpu")

In [7]:
torch.cuda.is_available()

True

In [8]:
torch.__version__

'1.11.0'

In [9]:
def set_seed(value):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)

In [10]:
set_seed(42)

## Carga de dataset

In [11]:
def to_tensor(obs, device, columns): 
    data = obs[columns]
    if type(data) == pd.DataFrame:
        data = data.values
    return torch.tensor(data).to(device)

features_fn = lambda obs, device: to_tensor(obs, device, ['user_seq', 'movie_seq'])
target_fn   = lambda obs, device: to_tensor(obs, device, ['user_movie_rating'])

dataset = ds.MovieLensTMDBDatasetFactory.from_path(
    path             = f'{BASE_PATH}/datasets',
    transform        = features_fn,
    target_transform = target_fn,
    device           = cpu,
    filter_fn        = lambda df: df[(df['user_movie_rating_year'] >= 2005) & (df['user_movie_rating_year'] <= 2019)]
)
dataset.info

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153892 entries, 0 to 153891
Data columns (total 16 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   user_id                      153892 non-null  int64         
 1   user_seq                     153892 non-null  int64         
 2   user_movie_tags              153892 non-null  object        
 3   user_movie_rating            153892 non-null  int64         
 4   user_movie_rating_timestamp  153892 non-null  datetime64[ns]
 5   user_movie_rating_year       153892 non-null  int64         
 6   movie_id                     153892 non-null  int64         
 7   movie_seq                    153892 non-null  int64         
 8   movie_title                  153892 non-null  string        
 9   movie_genres                 153892 non-null  object        
 10  movie_for_adults             153892 non-null  bool          
 11  movie_original_language   

In [12]:
train_set, eval_set = dataset.split_train_eval(split_year=2018)

2023-03-31 18:24:49,946 - INFO - Train: 82.85 % - Test: 8.97 %


## Definicion del modelo

In [13]:
params = Bunch({
    'lr': 0.001,
    'lr_factor': 0.1,
    'lr_patience': 3,
    'epochs': 50,
    'embedding_size': 50,
    'n_workers': 24,
    'batch_size': 64,
    'n_users': len(dataset.features_uniques[0]),
    'n_items': len(dataset.features_uniques[1]),
    'device': get_device()
})

In [14]:
train_dl = DataLoader(train_set, params.batch_size, num_workers=params.n_workers, pin_memory=True)
test_dl  = DataLoader(eval_set,  params.batch_size, num_workers=params.n_workers, pin_memory=True)

In [15]:
model = ml.BiasedGMF(
    n_users        = params.n_users,
    n_items        = params.n_items,
    embedding_size = params.embedding_size
).to(params.device)
model
# make_dot(y_pred, params=dict(model.named_parameters()))

KeyboardInterrupt: 

## Entrenamiento del modelo

In [ ]:
result = model.fit(
    train_dl,
    epochs      = params.epochs,
    loss_fn     = ml.MSELossFn(),
    optimizer   = Adam(
        params       = model.parameters(),
        lr           = params.lr
    ),
    callbacks   = [
        Validation(
            test_dl,
            metrics = { 'val_loss': ml.MSELossFn(float_result=True)},
            each_n_epochs=1
        ),
        ReduceLROnPlateau(metric='val_loss', mode='min', factor=params.lr_factor, patience=params.lr_patience),
        MetricsPlotter(metrics=['train_loss', 'val_loss'], plot_each_n_epochs=2),
        Logger(['time', 'epoch', 'train_loss', 'val_loss', 'patience', 'lr']),
        SaveBestModel(metric='val_loss', path='../weights', experiment_name='gfm_bias')
    ]
)

In [ ]:
PATH = f'{BASE_PATH}/weights/2022-07-23_16-23-41--gfm_bias--epoch_21--val_loss_0.9372825026512146.pt'
model.load_state_dict(torch.load(PATH))

deep_fm_predictor = ml.ModulePredictor(model)

validator = ml.Validator(
    n_samples  = 500,
    batch_size = 10000,
    metrics    = [
        mt.RMSE(),

        mt.MeanAveragePrecisionAtk(user_index=0, k=5, discretizer=dr.between(4, 5)),
        mt.MeanUserFBetaScoreAtk(user_index=0, k=5, discretizer=dr.between(4, 5)),
        mt.MeanUserPrecisionAtk(user_index=0, k=5, discretizer=dr.between(4, 5)),
        mt.MeanUserRecallAtk(user_index=0, k=5, discretizer=dr.between(4, 5)),
        
        mt.MeanAveragePrecisionAtk(user_index=0, k=5, discretizer=dr.between(3, 5)),
        mt.MeanUserFBetaScoreAtk(user_index=0, k=5, discretizer=dr.between(3, 5)),
        mt.MeanUserPrecisionAtk(user_index=0, k=5, discretizer=dr.between(3, 5)),
        mt.MeanUserRecallAtk(user_index=0, k=5, discretizer=dr.between(3, 5))
    ],
    predictors = [deep_fm_predictor]
)

summary = validator.validate(eval_set)
summary.save(f'{BASE_PATH}/metrics/biased_gmf')

In [ ]:
summary = ml.ValidatorSummary.load(f'{BASE_PATH}/metrics/biased_gmf')
summary.show()

In [ ]:
summary.plot(bins=10)